In [1]:
import warnings
from pathlib import Path
from loguru import logger
from datetime import datetime
from typing import Optional, Union
from itertools import chain

from astropy.io import fits
from astropy.io import votable
from astropy.io.votable import tree

import astropy.wcs as wcs
from astropy.coordinates import SkyCoord

import astropy.units as u
from astropy.nddata.utils import Cutout2D
from astropy.time import Time
from mocpy import MOC, STMOC

from vast_post_processing import crop, corrections

DATA_PATH = (Path(".") / "test-data" / "TILES").resolve()
IMAGE_PATH = (
    DATA_PATH
    / "STOKESI_IMAGES"
    / "epoch_38"
    / "image.i.VAST_0334-37.SB50801.cont.taylor.0.restored.conv.fits"
)
VOTABLE_PATH = (
    DATA_PATH
    / "STOKESI_SELAVY"
    / "epoch_38"
    / "selavy-image.i.VAST_0334-37.SB50801.cont.taylor.0.restored.conv.components.xml"
)

In [4]:
hdu = fits.open(IMAGE_PATH)[0]
field_centre = crop.get_field_centre(hdu.header)
cropped_hdu = crop.crop_hdu(hdu, field_centre)
vot = votable.parse(VOTABLE_PATH)
cropped_table = crop.crop_catalogue(vot, cropped_hdu)
# moc = crop.wcs_to_moc(cropped_hdu)
# stmoc = crop.moc_to_stmoc(moc, hdu)

2023-08-23 14:14:23.330 | DEBUG    | vast_post_processing.crop:get_field_centre:28 - Finding field centre
2023-08-23 14:14:23.338 | DEBUG    | vast_post_processing.crop:get_field_centre:35 - <SkyCoord (FK5: equinox=2000.0): (ra, dec) in deg
    (53.61702083, -37.63907778)>
2023-08-23 14:14:23.340 | DEBUG    | vast_post_processing.crop:crop_hdu:49 - Cropping HDU
2023-08-23 14:14:23.826 | DEBUG    | vast_post_processing.crop:crop_catalogue:72 - Cropping catalogue


In [21]:
from astropy.io.votable import parse
from astropy.io.votable.tree import Table

catalog_path = DATA_PATH / "STOKESI_SELAVY" / "epoch_38" / "test_catalog.xml"
votablefile = parse(catalog_path)
votable: Table = votablefile.get_first_table()

In [22]:
for param in votablefile.params:
    print(param.name, param.value)

print("=========")
votablefile = corrections.shift_and_scale_catalog(catalog_path)
for param in votablefile.params:
    print(param.name, param.value)

2023-08-23 14:34:17.852 | DEBUG    | vast_post_processing.corrections:shift_and_scale_catalog:301 - Correcting /home/hansen/projects/vast/projects/vast-post-processing/tests/test-data/TILES/STOKESI_SELAVY/epoch_38/test_catalog.xml ...


ra_offset 0.0
dec_offset 0.0
flux_scl 1.0
flux_offset 0.0
